In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
import bs4
import time
import random
import requests
import numpy as np
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from urllib.request import Request, urlopen
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
from selenium.webdriver.common.by import By
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver import Chrome, DesiredCapabilities, Remote, Firefox

In [21]:
class Scraper:
    
    def __init__(self):
        self.base_link = 'http://yocket.in/applications-admits-rejects'
        self.universities = {'mit':'27-massachusetts-institute-of-technology', 
                             'utd':'195-university-of-texas-dallas', 
                             'neu':'135-northeastern-university-boston', 
                             'usc':'168-university-of-southern-california', 
                             'ncsu': '31934-north-carolina-state-university-raleigh'
                             }
        self.pages = {'2': [1,113,51,67,40],'3': [1,90,83,48,59]}
        self.cols = ['GRE', 'IELTS', 'TOEFL', 'UNDERGRAD', 'WORK EX', 'YEAR', 'UNIVERSITY', 'STATUS']
        self.delays = [5,7,9,11,13]
        print("Creating links to hit")
        self.construct_links()
    
    def construct_links(self):
        self.all_links = []
        for option in self.pages:
            for ind,u in enumerate(self.universities):
                for p in range(self.pages[option][ind]):
                    if p == 0:
                        full_link = self.base_link+'/'+self.universities[u]+'/'+option
                    else:
                        full_link = self.base_link+'/'+self.universities[u]+'/'+option+'?page='+str(p+1)
                    tmp = {}
                    tmp['option'] = option
                    tmp['univ'] = u
                    tmp['page'] = str(p+1)
                    tmp['link'] = full_link
                    self.all_links.append(tmp)
        self.total_len = len(self.all_links)
    
    def scrape_page(self, link, driver):
        self.data = []
        driver.get(link)
        time.sleep(2)
        smalls = driver.find_elements_by_tag_name('small')
        divs = driver.find_elements_by_xpath("//div[@class='col-sm-3 col-xs-6']")
        yrs = [s.text.strip().split('\n')[1] if bool(re.search(r'\d', s.text)) else 'None' for s in smalls]
        scores = {}
        for i,div in enumerate(divs):
            text = div.text.strip()
            try:
                key, value = text.split('\n')[0], text.split('\n')[1]
            except:
                key = text.split('\n')[0]
                value = 'NA'
            if i%4 == 0 and i!=0:
                tmp = {}
                tmp['year'] = yrs[i//4 - 1]
                tmp['score'] = scores
                self.data.append(tmp)
                scores = {}
            if 'N' in value and 'A' in value:
                scores[key] = 'NA'
            else:
                try:
                    scores[key] = value.split()[0]
                except:
                    scores[key] = value
        print(len(self.data))
            

    def collector(self, start=0, end=1):
        self.final_data = []
        
        driver = Firefox()
        driver.get('https://yocket.in/account/login')
        driver.find_element_by_name("email").send_keys("raghavsagar1997@gmail.com")
        cont_button = driver.find_elements_by_xpath("//div[@class='mt-4']")[0]
        cont_button.click()
        time.sleep(5)
        driver.find_element_by_name("password").send_keys("Raghav13")
        cont_button = driver.find_elements_by_xpath("//div[@class='mt-4']")[0]
        cont_button.click()
        
        for tmp in self.all_links[start:end]:
            print(tmp['link'])
            time.sleep(self.delays[random.randint(0,len(self.delays)-1)])
            self.scrape_page(tmp['link'], driver)
            for d in self.data:
                if d['year'] == 'None':
                    continue
                if not all(item in self.cols for item in d['score']):
                    continue
                tup = [None]*8
                scores = d['score']
                for k in scores:
                    if scores[k] == 'NA' and k!='WORK EX':
                        tup[self.cols.index(k)] = None
                    elif scores[k] == 'NA' and k=='WORK EX':
                        tup[self.cols.index(k)] = 0.0
                    else:
                        tup[self.cols.index(k)] = float(scores[k])
                tup[-3] = d['year']
                tup[-2] = tmp['univ']
                tup[-1] = tmp['option']
                self.final_data.append(tuple(tup))
        
        name = 'data_'+str(start)+'_to_'+str(end)+'.csv'
        self.save_data(name)
        driver.quit()
    
    def save_data(self, name):
        df = pd.DataFrame(self.final_data, columns=self.cols)
        df.to_csv(name, index=None, sep=',')

In [23]:
begin = 550
end = 555
skip = 5

scraper = Scraper()

for start in range(begin,end,skip):
    scraper.collector(start,start+skip)
    print("Processed {} Done {}".format(len(scraper.final_data),start+skip))

Creating links to hit
http://yocket.in/applications-admits-rejects/31934-north-carolina-state-university-raleigh/3?page=57
20
http://yocket.in/applications-admits-rejects/31934-north-carolina-state-university-raleigh/3?page=58
20
http://yocket.in/applications-admits-rejects/31934-north-carolina-state-university-raleigh/3?page=59
20
Processed 59 Done 555
